# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902049


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/33 [00:00<?, ?it/s]

Rendering models:   6%|▌         | 2/33 [00:02<00:43,  1.40s/it]

Rendering models:   9%|▉         | 3/33 [00:02<00:30,  1.01s/it]

Rendering models:  12%|█▏        | 4/33 [00:03<00:21,  1.35it/s]

Rendering models:  15%|█▌        | 5/33 [00:03<00:15,  1.82it/s]

...

Rendering models:  82%|████████▏ | 27/33 [00:09<00:01,  5.70it/s]

Rendering models:  85%|████████▍ | 28/33 [00:09<00:00,  6.44it/s]

Rendering models:  88%|████████▊ | 29/33 [00:09<00:00,  7.01it/s]

Rendering models: 100%|██████████| 33/33 [00:10<00:00,  8.84it/s]

equidad1                              0.000119
StephAChamber                         0.000078
Wain-PalmerM                          0.000069
not-logged-in-0c20e1f083ae2ffe33bd    0.000064
not-logged-in-8c0c44faf754d344b18a    0.000094
Thedunkmasta                          0.000300
citystars                             0.000088
not-logged-in-20487e5808245554c355    0.000089
not-logged-in-efcb79850891fdd4daf8    0.000075
DavidCastillo1702                     0.000212
lsautter                              0.000103
FelipedaFonseca                       0.000129
not-logged-in-5f30b2bb9b7f9d49da41    0.017017
Planetme                              0.000608
acapirala                             0.000082
aidenr                                0.000433
haydenmanninen                        0.000078
chavez01                              0.000186
Lavadude                              0.027770
layne.woodward2701                    0.000065
not-logged-in-15498ebaa693dd5a198d    0.000085
w7250369     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())